In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


In [5]:
cust_df = pd.read_csv("/Users/Dell/Documents/college/Projects/Customer_Segmentation/Neptune_ai /customers-data - customers-data.csv")

In [6]:
cust_df.head()

,customer_id,products_purchased,complains,money_spent
0,1000661,1,0,260.0
1,1001914,1,0,790.2
2,1002167,3,0,234.2
3,1002387,1,0,890.0
4,1002419,2,0,103.0


In [7]:
cust_df.shape

(24000, 4)

In [8]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         24000 non-null  int64  
 1   products_purchased  24000 non-null  int64  
 2   complains           24000 non-null  int64  
 3   money_spent         24000 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 750.1 KB


In [9]:
from sklearn.cluster import KMeans 
import plotly.express as px
import plotly.graph_objects as go 


In [10]:
kmeans_model = KMeans(init = 'k-means++', max_iter = 400, random_state = 42)

In [11]:
kmeans_model.fit(cust_df[['products_purchased', 'complains', 'money_spent']])

/Users/Dell/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(max_iter=400, random_state=42)

In [12]:
def try_different_clusters(K,data):
    cluster_values = list(range(1,K+1))
    inertias = []
    
    for c in cluster_values:
        model = KMeans(n_clusters = c, init = 'k-means++', max_iter = 400, random_state = 42)
        model.fit(data)
        inertias.append(model.inertia_)
    
    return inertias 

In [13]:
outputs = try_different_clusters(12, cust_df[['products_purchased', 'complains', 'money_spent']])
distances = pd.DataFrame({"clusters": list(range(1,13)), "sum of squared distances": outputs})

/Users/Dell/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Dell/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Dell/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/Dell/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicit

In [14]:
figure = go.Figure()
figure.add_trace(go.Scatter(x= distances["clusters"], y = distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1, dtick = 1, tickmode = 'linear'), 
                    xaxis_title = "Number of clusters",
                    yaxis_title = "Sum of squared distances", 
                    title_text = "Finding optimal number of clusters using elbow method ")

import plotly.io as pio
pio.renderers.default = 'iframe'

figure.show()

In [15]:
kmeans_model_new = KMeans(n_clusters = 4, init = 'k-means++', max_iter = 400, random_state = 42)

kmeans_model_new.fit_predict(cust_df[['products_purchased', 'complains', 'money_spent']])

/Users/Dell/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([3, 1, 3, ..., 3, 2, 0], dtype=int32)

In [16]:
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data,cluster_centers, axis = 1)
points

/var/folders/8b/ft1wzckn60vd__kv3pxwy4km0000gn/T/ipykernel_12533/3138824467.py:2: RuntimeWarning:

overflow encountered in expm1



array([[6.38905610e+000, 5.01252086e-003, 1.14301649e+045,
        2.00000000e+000, 5.00000000e-003, 1.03750000e+002],
       [1.71828183e+000, 1.00501671e-002,             inf,
        1.00000000e+000, 1.00000000e-002, 7.92700000e+002],
       [1.71828183e+000, 2.02013400e-002,             inf,
        1.00000000e+000, 2.00000000e-002, 8.91750000e+002],
       [6.38905610e+000, 7.52819544e-003, 4.94508943e+107,
        2.00000000e+000, 7.50000000e-003, 2.47975000e+002]])

In [17]:
points = np.append(points,[[0],[1],[2],[3]], axis = 1)
cust_df["clusters"] = kmeans_model_new.labels_

In [18]:
cust_df.head()

,customer_id,products_purchased,complains,money_spent,clusters
0,1000661,1,0,260.0,3
1,1001914,1,0,790.2,1
2,1002167,3,0,234.2,3
3,1002387,1,0,890.0,2
4,1002419,2,0,103.0,0


In [19]:
figure = px.scatter_3d(cust_df, 
                      color = "clusters",
                      x="products_purchased",
                       y = "complains",
                       z = "money_spent",
                       category_orders = {"clusters": ["0","1","2","3"]}
                      )

figure.update_layout()
figure.show()